# Prospecção de Dados (Data Mining) DI/FCUL - HA3

## Third Home Assignement (MC/DI/FCUL - 2024)

### Fill in the section below

### GROUP: `13`

* Miguel Landum, 35019 - Hours worked on the project
* Niklas Schmitz, 62689 - Hours worked on the project
* Pol Parra, 62692 - Hours worked on the project
* Til Dietrich, 62928 - Hours worked on the project




The purpose of this Home Assignment is
* Find similar items with Local Sensitivity Hashing
* Do Dimensionality Reduction

**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**

**NOTE 3: Name of notebook should be: HA3_GROUP-XX.ipynb (where XX is the group number)**


**NOTE to run code locally:** add data (data_d3.pickle, data_d4.pickle) to **assignment-3/data/** folder

## 1. Read the Dataset

The dataset correspond to about 99% of the Human Proteome (set of known Human Proteins - about 19,500), coded with specific structural elements. They are presented in a dictionary where the key is the [UniprotID](https://www.uniprot.org/) of the protein and the value is a set of indices of a specific structural characteristic

Students can use one of two datasets, that are **not** subsets of each other: 
* `data_d3.pickle` - smaller set of structural features (2048)
* `data_d4.pickle` - much larger set of structural features (20736) **Note:** This dataset has been Zipped to fit into moodle. Students should unzip it before usage 

Select **one** of the datasets and perform all analyses with it. 

It may be adviseable the usage of sparse matrices, especially for the `d4` dataset



In [83]:
# !pip install datasketch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [1]:
### Your code Here
import pickle
from itertools import islice

data_d3=pickle.load(open("data/data_d3.pickle", "rb"))
data_d4=pickle.load(open("data/data_d4.pickle", "rb"))

first_elements = list(islice(data_d3, 5))
print(first_elements[0])
print(data_d3[first_elements[0]])
print(len(data_d3[first_elements[0]]))
print(len(data_d3[first_elements[1]])) # => elements in dict have different amount of elements in them

A0A024R1R8
{1, 771, 775, 263, 137, 11, 531, 1176, 922, 30, 927, 1185, 673, 675, 423, 296, 1704, 2043, 813, 304, 48, 52, 693, 1080, 1083, 1211, 1085, 1469, 69, 454, 1865, 458, 842, 1609, 1618, 467, 1109, 214, 1237, 473, 90, 603, 1379, 229, 1768, 1513, 1260, 109, 1645, 1007, 112, 1521, 242, 2035, 1652, 1525, 1270, 119, 1017, 123, 508, 1149}
62
178


## 2. Perform Local Sensitivity Hashing (LSH)

* examine the selected dataset in terms of similarities and select a set of LSH parameters able to capture the most similar proteins
* Comment your results

**BONUS POINTS:** It might be interesting to identify **some** of the candidate pairs in Uniprot, to check if they share some of the same properties (e.g. for [protein P28223](https://www.uniprot.org/uniprotkb/P28223/entry))


In [2]:
### Add supporting functions here
import numpy as np
from time import time
from scipy.sparse import csr_matrix
from datasketch import MinHash, MinHashLSH

def convert_to_binary_matrix(data):
    # Extract all unique values to create the universe of elements
    all_values = set()
    for values in data.values():
        all_values.update(values)

    # Create a mapping from values to index
    value_to_index = {value: idx for idx, value in enumerate(all_values)}

    # Number of unique elements
    num_elements = len(all_values)

    # Prepare data for csr_matrix
    rows = []
    cols = []
    data_values = []

    for key, values in data.items():
        for value in values:
            rows.append(key)
            cols.append(value_to_index[value])
            data_values.append(1)  # Presence of the element

    # Convert document IDs to numerical indices
    keys_to_index = {doc: idx for idx, doc in enumerate(data.keys())}
    row_indices = [keys_to_index[doc] for doc in rows]

    # Create and return CSR matrix
    return csr_matrix((data_values, (row_indices, cols)), shape=(len(data), num_elements)), keys_to_index

# Create MinHash signatures from sparse matrix
def create_minhash_from_sparse(matrix, num_perm):
    minhashes = []
    for i in range(matrix.shape[0]):
        minhash = MinHash(num_perm=num_perm)
        # reinitialization of minhash is not a problem as the library ensures that the same hash functions are used as long as num_perm stays the same value
        for idx in matrix[i].indices:
            minhash.update(idx)
        minhashes.append(minhash)
    return minhashes

def init_minhash_lsh(num_perm, threshold):
    return MinHashLSH(threshold=threshold, num_perm=num_perm)

In [3]:
csr_data, data_indices = convert_to_binary_matrix(data_d3)
# Number of permutations
num_perm = 8

# Generate MinHash signatures from sparse matrix
minhashes = create_minhash_from_sparse(csr_data, num_perm)

# Define threshold
threshold = 0.5
lsh = init_minhash_lsh(num_perm, threshold)

# Populate lsh with calculated MinHash signatures
for doc, minhash in zip(data_d3.keys(), minhashes):
    lsh.insert(doc, minhash)
    
print("\nLSH index created and signatures inserted.")

# Example Query: Find similar items to specific item key: P28223 => Bonus question
query_key = "P28223"
query_index = data_indices[query_key]
results = lsh.query(minhashes[query_index])
print(f"Items similar to {query_key}: {results}")
len(results)


LSH index created and signatures inserted.


In [13]:
# TODO all the stuff that I did here is nice and solves the task but he wants
# an implementation where we have more control => use the one from the TP: LSHT
# Convert the data to binary can be kept and is definetily necessary
# Code can be kept to answer the bonus question

### Your short analysis here

Plan:
- use sparse matrices!
- use MinHashing to calculate the Jaccard Similarity as it is a good approach for large datasets (like d4)
- use LSHT instead of LSH as it heavily outperformes LSH for sparse matrices (which is the case for this data)


## 3. Do dimensionality reduction 

Use the techniques discussued in class to make an appropriate dimensional reduction of the selected dataset. It is not necesary to be extensive, **it is better to select one approach and do it well than try a lot of techniques with poor insights and analysis**

It is important to do some sensitivity analysis, relating the dataset size reduction to the loss of information



In [ ]:
### Add supporting functions here



In [ ]:
### Add processing code here



## 4. Discuss your findings [to fill on your own]

* Comment your results above
* Discuss how could they be used for the full Uniprot that currently has about [248 Million proteins](https://www.uniprot.org/uniprotkb/statistics)


Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum
